In [ ]:
%%capture
%run "main.ipynb"

In [ ]:
from collections import Counter
from sklearn.preprocessing import TargetEncoder
import numpy as np
import json

In [ ]:
def cumulatively_categorise(column,threshold=0.75):
    threshold_value=int(threshold*len(column))
    categories_list=[]
    s=0
    counts=Counter(column)
    for i,j in counts.most_common():
        s+=dict(counts)[i]
        categories_list.append(i)
        if s>=threshold_value:
            break
    categories_list.append('Other')
    new_column=column.apply(lambda x: x if x in categories_list else 'Other')
    return pd.Series(new_column)

# Data cleaning

Er zijn 2 rijen die een prioriteit missen, deze twee rijen halen we uit de database.

In [ ]:
# NA's verwijderen omdat het er weinig zijn
df = df.dropna(subset=['stm_prioriteit'])

Er zijn 5 rijen die een geocode missen. Ook deze halen wij uit de database. \
Verder vullen wij voor missende oorzaakcodes, de code 999. 
Deze wordt in de database al gebruikt, en is niet beschreven in de data dictionary die wij hebben gekregen.
Het is een default/missende waarde. \
Voor het contractgebied vervangen wij missende waardes met 999. Deze komt in de dataset niet voor en is vergeleken met de andere waardes in deze kolom, overduidelijk een default/overig value.

In [ ]:
# stm_geo_mld
# Some computers differ in how they interpret a column without values
# So we remove both NaN values and empty strings
df = df[df.stm_geo_mld != '']
df = df.dropna(subset=['stm_geo_mld'])
df['stm_geo_mld'] = df['stm_geo_mld'].astype(float).astype(int).astype(str)

In [ ]:
# stm_oorz_code
df['stm_oorz_code'] = df['stm_oorz_code'].fillna(221).astype(int).astype(str)

In [ ]:
# stm_techn_mld
df['stm_techn_mld'] = df['stm_techn_mld'].replace('', 'X')
df['stm_techn_mld'] = df['stm_techn_mld'].fillna('X')

In [ ]:
# stm_contractgeb_mld
df['stm_contractgeb_mld'] = df['stm_contractgeb_mld'].fillna(999)
df['stm_contractgeb_mld'] = df['stm_contractgeb_mld'].astype(float).astype(int).astype(str)

In [ ]:
# stm_oorz_groep
df['stm_oorz_groep'] = df['stm_oorz_groep'].replace('', 'ONBK', regex=True)

In [ ]:
# stm_progfh_in_duur
df['stm_progfh_in_duur'] = df['stm_progfh_in_duur'].str.strip('*- ').astype(float).astype(int)

# Data preparation

In [ ]:
# Setup TargetEncoder
tEnc = TargetEncoder(target_type="continuous", random_state=42)
y = df['anm_tot_fh']

## stm_prioriteit

Deze variabele gebruiken wij als een ordinale meetwaarde en hoeft dus niet veranderd te worden.

df['stm_prioriteit'].info()
prioriteit_dummies = pd.get_dummies(df['stm_prioriteit'], prefix='prio')
prioriteit_dummies.info()

## stm_oorz_code

In [ ]:
X = np.array(df['stm_oorz_code']).reshape(-1, 1)
tEnc.fit(X,y)
df['oorz_code_enc'] = tEnc.transform(X)

In [ ]:
oorzc_dict = {}
for i in range(len(tEnc.categories_[0])):
    cat, enc = tEnc.categories_[0][i], tEnc.encodings_[0][i]
    oorzc_dict[cat] = enc
# oorzc_dict

## stm_geo_mld

In [ ]:
X = np.array(df['stm_geo_mld']).reshape(-1, 1)
tEnc.fit(X,y)
df['geo_code_enc'] = tEnc.transform(X)

In [ ]:
geo_dict = {}
for i in range(len(tEnc.categories_[0])):
    cat, enc = tEnc.categories_[0][i], tEnc.encodings_[0][i]
    geo_dict[cat] = enc
# geo_dict

## stm_contractgeb_mld

In [ ]:
X = np.array(df['stm_contractgeb_mld']).reshape(-1, 1)
tEnc.fit(X,y)
df['contractgb_enc'] = tEnc.transform(X)

In [ ]:
contrgb_dict = {}
for i in range(len(tEnc.categories_[0])):
    cat, enc = tEnc.categories_[0][i], tEnc.encodings_[0][i]
    contrgb_dict[cat] = enc
# contrgb_dict

## stm_techn_mld

In [ ]:
X = np.array(df['stm_techn_mld']).reshape(-1, 1)
tEnc.fit(X,y)
df['techn_veld_enc'] = tEnc.transform(X)

In [ ]:
techn_dict = {}
for i in range(len(tEnc.categories_[0])):
    cat, enc = tEnc.categories_[0][i], tEnc.encodings_[0][i]
    techn_dict[cat] = enc
# techn_dict

In [ ]:
techn_veld_dummies = pd.get_dummies(df['stm_techn_mld'], prefix='techn_veld')

## stm_oorz_groep

In [ ]:
oorz_groep_dummies = pd.get_dummies(df['stm_oorz_groep'], prefix='oorzgr')

## stm_progfh_in_duur

Deze kolom is boven omgezet naar integer waardes, en heeft verder geen preparatie nodig.

# Geprepareerde data

In [ ]:
df = df.reset_index(drop=True)
# prioriteit_dummies = prioriteit_dummies.reset_index(drop=True)
prioriteit = df['stm_prioriteit'].reset_index(drop=True)
oorz_code_enc = df['oorz_code_enc'].reset_index(drop=True)
geo_code_enc = df['geo_code_enc'].reset_index(drop=True)
contractgb_enc = df['contractgb_enc'].reset_index(drop=True)
techn_veld_enc = df['techn_veld_enc'].reset_index(drop=True)
techn_veld_dummies = techn_veld_dummies.reset_index(drop=True)
oorz_groep_dummies = oorz_groep_dummies.reset_index(drop=True)
fh_prog = df['stm_progfh_in_duur']

features_to_use = [
    # prioriteit_dummies
    prioriteit,
    oorz_code_enc,
    geo_code_enc,
    contractgb_enc,
    techn_veld_enc,
    # techn_veld_dummies,
    # oorz_groep_dummies,
    fh_prog
]

model_df = pd.concat([df['anm_tot_fh'], *features_to_use], axis=1)
# model_df = model_df.dropna()
model_df.sample(20)

In [ ]:
model_df.info()

In [ ]:
model_df.to_pickle("data/model_df.pkl")

In [ ]:
encodings = {'oorz_code': oorzc_dict,
 'geo_code': geo_dict,
 'contractgb': contrgb_dict,
 'techn_veld': techn_dict
}
with open('data/feature_encodings.json', 'w') as outfile:
    json.dump(encodings, outfile)